In [165]:
#coding:utf-8
from numpy import *

#创建数据
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #标签：1 is 侮辱性文字, 0 不是
    return postingList,classVec

#创建一个不重复的词列表
def createVocabList(dataSet):
    vocabSet = set([])#创建一个空集
    for document in dataSet:
        vocabSet = vocabSet | set(document)#创建两个集合的并集
    return list(vocabSet)

def setOfWords2Vec(vocabList,inputSet):
    returnVec = [0]*len(vocabList)#创建一个和集合相同长度，全为0的向量
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

#朴素贝叶斯分类器训练函数
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)#计算p(1) p(0)等于1-p(1)但是对于多充情况分类需要修改算法
    p0Num = ones(numWords);p1Num = ones(numWords)
    #初始化概率
    p0Denom = 2.0;p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]#向量相加
            p1Denom += sum(trainMatrix[i])#数值相加
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom) #在已知侮辱性文档的条件下，对应词的概率向量
    p0Vect = log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    
def testingNB():
    list0Posts,listClasses = loadDataSet()
    myVocabList = createVocabList(list0Posts)
    trainMat = []
    for postingDoc in list0Posts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    
    testEntry = ['love','my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
    
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    
    
#文件解析及完整的垃圾邮件测试函数
def textParse(bigString):    #input is big string, #output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 

def spamTest():
    docList=[];classList = [];fullText = []
    for i in range(1,26):#将整个文件夹的文件全部导入并解析
        wordList = textParse(open('E:/email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('E:/email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
        
    vocabList = createVocabList(docList)
    trainingSet = list(range(50));testSet = []#构建训练集和测试集
    
    for i in range(10):#随机构建训练集
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])#在trianingSet中随机找出10个数作为测试集索引，并将对应的训练集中的索引删除
        del(trainingSet[randIndex])
    trianMat =[];trainClasses=[]
    for docIndex in trainingSet:
        trianMat.append(setOfWords2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trianMat),array(trainClasses))
    
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList,docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error",docList[docIndex])
    print('the error rate is : ',float(errorCount)/len(testSet))
            

In [5]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
myVocabList

['love',
 'take',
 'him',
 'licks',
 'so',
 'ate',
 'park',
 'stop',
 'worthless',
 'quit',
 'I',
 'garbage',
 'dog',
 'steak',
 'cute',
 'to',
 'stupid',
 'has',
 'please',
 'mr',
 'not',
 'my',
 'problems',
 'posting',
 'help',
 'how',
 'flea',
 'maybe',
 'buying',
 'is',
 'food',
 'dalmation']

In [11]:
setOfWords2Vec(myVocabList,listOPosts[0])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [14]:
list0Posts,listClasses = loadDataSet()
list0Posts

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [15]:
listClasses

[0, 1, 0, 1, 0, 1]

In [19]:
myVocablist = createVocabList(list0Posts)
myVocablist

['love',
 'take',
 'him',
 'licks',
 'so',
 'ate',
 'park',
 'stop',
 'worthless',
 'quit',
 'I',
 'garbage',
 'dog',
 'steak',
 'cute',
 'to',
 'stupid',
 'has',
 'please',
 'mr',
 'not',
 'my',
 'problems',
 'posting',
 'help',
 'how',
 'flea',
 'maybe',
 'buying',
 'is',
 'food',
 'dalmation']

In [21]:
trainMat = []
#将每一个文档的向量组合成一个完整的向量
for postinDoc in list0Posts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
trainMat

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0],
 [0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0],
 [1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0]]

In [25]:
trainMat[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [37]:
p0V,p1V,pAb = trainNB0(trainMat,listClasses)
pAb

0.5

In [48]:
p0V

array([-2.56494936, -3.25809654, -2.15948425, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -2.56494936, -3.25809654, -3.25809654,
       -2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -3.25809654, -1.87180218, -2.56494936, -3.25809654, -2.56494936,
       -2.56494936, -2.56494936, -3.25809654, -3.25809654, -2.56494936,
       -3.25809654, -2.56494936])

In [47]:
p1V

array([-3.04452244, -2.35137526, -2.35137526, -3.04452244, -3.04452244,
       -3.04452244, -2.35137526, -2.35137526, -1.94591015, -2.35137526,
       -3.04452244, -2.35137526, -1.94591015, -3.04452244, -3.04452244,
       -2.35137526, -1.65822808, -3.04452244, -3.04452244, -3.04452244,
       -2.35137526, -3.04452244, -3.04452244, -2.35137526, -3.04452244,
       -3.04452244, -3.04452244, -2.35137526, -2.35137526, -3.04452244,
       -2.35137526, -3.04452244])

In [60]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as:  0


In [152]:
spamTest()

the error rate is :  0.1


D:\develop\Anaconda\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [166]:
spamTest()

classification error ['been', 'working', 'running', 'website', 'using', 'jquery', 'and', 'the', 'jqplot', 'plugin', 'not', 'too', 'far', 'away', 'from', 'having', 'prototype', 'launch', 'you', 'used', 'jqplot', 'right', 'not', 'think', 'you', 'would', 'like']
classification error ['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today']
the error rate is :  0.2


D:\develop\Anaconda\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
